In [1]:
import json
f = open("./qtableeasier.json", "x")

In [2]:
!pip install chess

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.0/148.0 kB 350.9 kB/s eta 0:00:00


In [3]:
import chess
import numpy as np
q={}

In [4]:
def convert_moves_list (legalMoves):
  moves = str(legalMoves)
  return moves[moves.index('(')+1:moves.index(')')].split(', ')


In [5]:
def checkDone (board):
  return board.is_checkmate() or board.is_stalemate() or board.is_insufficient_material()

In [6]:
def giveReward (board):
  if not checkDone(board): return -5
  if board.is_checkmate():
    return 100
  if board.is_stalemate() or board.is_insufficient_material():
    return -100

In [7]:
def addKeys (board):
  if hash(board) not in q.keys():
    q[hash(board)] = {}
  valid_moves = convert_moves_list(board.legal_moves)
  for i in valid_moves:
    if i not in q[hash(board)].keys():
      q[hash(board)][i] = 0


In [8]:
def random_agent (board):
  valid_moves = convert_moves_list(board.legal_moves)
  board.push_san(np.random.choice(valid_moves))
  return board

In [9]:
def hash(board):
  return board.fen()

In [10]:
def game(startingFen):
  
  board = chess.Board(startingFen)
  moveNumber = 0
  while True:
    moveNumber+=1
    if moveNumber > 50: return
    gamma = 0.8
    alpha = 0.2
    epsilon = 0.4
    addKeys(board)
    #print(board.legal_moves)
    valid_moves = convert_moves_list(board.legal_moves)
    move = ""
    reward = 0

    if np.random.uniform(0, 1)< epsilon:
        move= np.random.choice(valid_moves)
    else:
      move = max(zip(q[hash(board)].values(), q[hash(board)].keys()))[1]
    newboard = board.copy()
    newboard.push_san(move)
    reward += giveReward(newboard)
    if not checkDone(newboard):
      newboard = random_agent(newboard)
    addKeys(newboard)
    q[hash(board)][move] = (1-alpha)*q[hash(board)][move] + alpha * (reward + gamma*max(q[hash(newboard)].values()))  
    board = newboard
    if checkDone(board): return

In [11]:

successes = []
def testGame(startingFen):
  
  board = chess.Board(startingFen)
  moveNumber = 0
  while True:
    if hash(board) not in q.keys():
      successes.append(0)
      return
    moveNumber+=1
    if moveNumber > 50: 
      successes.append(0)
      return
    move = max(zip(q[hash(board)].values(), q[hash(board)].keys()))[1]
    board.push_san(move)
    if not checkDone(board):
      board = random_agent(board)
    else:
      if board.is_checkmate():
        successes.append(1)
      else:
        successes.append(0)
      return

In [12]:
games = 100*1000
fen = 'k7/8/8/8/3QK3/8/8/8 w - - 8 5'
for i in range (games):
  game(fen)
  if i%(games/100)==0:print(i/games*100) 

0.0
1.0
2.0
3.0
4.0
5.0
6.0
7.000000000000001
8.0
9.0
10.0
11.0
12.0
13.0
14.000000000000002
15.0
16.0
17.0
18.0
19.0
20.0
21.0
22.0
23.0
24.0
25.0
26.0
27.0
28.000000000000004
28.999999999999996
30.0
31.0
32.0
33.0
34.0
35.0
36.0
37.0
38.0
39.0
40.0
41.0
42.0
43.0
44.0
45.0
46.0
47.0
48.0
49.0
50.0
51.0
52.0
53.0
54.0
55.00000000000001
56.00000000000001
56.99999999999999
57.99999999999999
59.0
60.0
61.0
62.0
63.0
64.0
65.0
66.0
67.0
68.0
69.0
70.0
71.0
72.0
73.0
74.0
75.0
76.0
77.0
78.0
79.0
80.0
81.0
82.0
83.0
84.0
85.0
86.0
87.0
88.0
89.0
90.0
91.0
92.0
93.0
94.0
95.0
96.0
97.0
98.0
99.0


In [13]:
for i in range (100):
  testGame(fen)
sum(successes)/100

1.0

In [14]:
with open('./qtableeasier.txt', 'w') as convert_file:
     convert_file.write(json.dumps(q))